# Homework 5
Evelina Teran & Kevin Smith

In [ ]:
import numpy as np
import pandas as pd
import platform
import os
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
# Lorenz Loss:
# if(x >1): 0
# else: ln(1+(x-1)**2)

## Question 1

In [ ]:
data_direc = os.getcwd() + "/"
if platform.system() == "Windows":
    data_direc = data_direc.replace("/", "\\")
train = pd.read_csv(data_direc + "gisette_train.csv")
train_labels = pd.read_csv(data_direc + "gisette_train_labels.csv")
test = pd.read_csv(data_direc + "gisette_valid.csv")
test_labels = pd.read_csv(data_direc + "gisette_valid_labels.csv")
train = np.delete(train, 5000, axis=1)
test = np.delete(test, 5000, axis=1)

In [19]:
# Normalize the data
scaler = StandardScaler()
train_normalized = scaler.fit_transform(train)
test_normalized = scaler.transform(test)

# Define parameters
s = 0.001
mu = 300
N_iter = 300

# Define k values
k_values = [10, 30, 100, 300, 500]

# Initialize beta, gamma, and selected variables
beta = np.zeros(train_normalized.shape[1])
gamma = np.zeros(train_normalized.shape[1])
selected_vars = []

# FSA algorithm with modification to record the loss
losses_k100 = []  # To store the training loss for k=100
for _ in range(N_iter):
    loss = 0  # Initialize loss for this iteration
    for j in range(train_normalized.shape[1]):
        gamma_j = gamma[j] + mu * beta[j]
        beta_j = beta.copy()
        beta_j[j] = 0
        z = np.dot(train_normalized, beta_j)
        yz = np.multiply(train_labels.values.flatten(), z)
        l_prime = np.where(yz > 1, 0, -2 * (yz - 1) / (1 + (yz - 1) ** 2))
        gradient_j = np.mean(np.multiply(l_prime, train_normalized[:, j])) + 2 * s * beta[j]
        gamma[j] = gamma_j - mu * gradient_j
        beta[j] = np.sign(gamma[j]) * max(0, abs(gamma[j]) - mu * s)
        if (yz <= 1).all():
            loss += np.log(1 + (yz - 1) ** 2)
    
    losses_k100.append(loss / len(train_normalized))

# Plot training loss vs iteration number for k=100
plt.plot(losses_k100)
plt.xlabel('Iteration number')
plt.ylabel('Training loss')
plt.title('Training loss vs iteration number for k=100')
plt.show()

# Initialize lists to store results
train_errors = []
test_errors = []
roc_aucs = []

# Plot misclassification error vs k
plt.plot(k_values, train_errors, label='Training error')
plt.plot(k_values, test_errors, label='Test error')
plt.xlabel('Number of features (k)')
plt.ylabel('Misclassification error')
plt.title('Misclassification error vs k')
plt.legend()
plt.show()

# Define k value for ROC curve
k_roc = 100

# Select the top k features based on the FSA algorithm for ROC curve
selected_vars_k100 = np.argsort(np.abs(gamma))[-k_roc:]
train_selected_k100 = train[:, selected_vars_k100]
test_selected_k100 = test[:, selected_vars_k100]

# Use the existing logistic regression model with k=100 features for predictions
train_pred_prob_k100 = np.dot(train_selected_k100, beta)
test_pred_prob_k100 = np.dot(test_selected_k100, beta)

# Calculate the false positive rate (fpr) and true positive rate (tpr) for the ROC curve
fpr_k100, tpr_k100, _ = roc_curve(test_labels, test_pred_prob_k100)

# Calculate the ROC AUC score
roc_auc_k100 = auc(fpr_k100, tpr_k100)

# Plot ROC curve for k=100
plt.figure()
plt.plot(fpr_k100, tpr_k100, label='ROC curve (area = %0.2f)' % roc_auc_k100)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for k=100')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Assume train and test data are already defined
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# Define k values
k_values = [10, 30, 100, 300, 500]

# Perform FSA
train_errors, test_errors, train_roc_auc, test_roc_auc, fpr_train, tpr_train, fpr_test, tpr_test = fsa(train, train_labels, test, test_labels, k_values)

# Plot ROC curves
num_features_values = [10, 30, 100, 300, 500]
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

for i, num_features in enumerate(num_features_values):
    ax = axs[i // 3, i % 3]
    ax.plot(fpr_train[i], tpr_train[i], label="Training ROC Curve (AUC = {:.2f})".format(train_roc_auc[i]))
    ax.plot(fpr_test[i], tpr_test[i], label="Test ROC Curve (AUC = {:.2f})".format(test_roc_auc[i]))
    ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title(f"ROC Curve for {num_features} features")
    ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

## Question 2

In [ ]:
train = pd.read_csv(data_direc + "dexter_train.csv", header = None)
train_labels = pd.read_csv(data_direc + "dexter_train_labels.csv", header = None)
test = pd.read_csv(data_direc + "dexter_valid.csv", header = None)
test_labels = pd.read_csv(data_direc + "dexter_valid_labels.csv", header = None)

In [ ]:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

## Question 3

In [ ]:
train = pd.read_fwf(data_direc + "madelon_train.data", header = None)
train_labels = pd.read_fwf(data_direc + "madelon_train.labels", header = None)
test = pd.read_fwf(data_direc + "madelon_valid.data", header = None)
test_labels = pd.read_fwf(data_direc + "madelon_valid.labels", header = None)

In [ ]:
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)